In [ ]:
import pandas as pd
import datetime
from waipawama import Finance
import pathlib
import math
import calendar
import datetime as dt
import redis

import pyarrow as pa
import pyarrow.parquet as pq

from mlrepricer import helper
%alias_magic t timeit

In [ ]:
r = redis.Redis(**helper.rediscred, decode_responses=True)

In [ ]:
lua_script = """
local result = 0
for l, atomic in ipairs(redis.call('SMEMBERS', KEYS[1]))
do result = result + redis.call('HGET', 'atomicID:' .. atomic, 'amount')
end
return result
"""

In [ ]:
result = r.eval(lua_script, 1, 'account:atomic:4400.0')

In [ ]:
r.hgetall('generalID:10')

In [ ]:
result